In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

In [2]:
# Define paths to images and masks folders
images_dir = r"C:\Users\User\Desktop\ownrepo\Dataset\train"
masks_dir = r"C:\Users\User\Desktop\ownrepo\Masks"

In [3]:
# Function to load data
def load_data(images_dir, masks_dir):
    images = []
    masks = []
    class_labels = {"root": 0, "seed": 1, "shoot": 2, "occluded_root": 3}
    
    mask_files = os.listdir(masks_dir)
    
    for img_file in os.listdir(images_dir):
        if img_file.endswith(".png"):
            img_path = os.path.join(images_dir, img_file)
            corresponding_mask_file = None
            
            for mask_file in mask_files:
                if mask_file.startswith(img_file[:-4]):
                    corresponding_mask_file = mask_file
                    break
            
            if corresponding_mask_file:
                mask_path = os.path.join(masks_dir, corresponding_mask_file)
                class_name = corresponding_mask_file.split("_")[-2]  # Extract class name from filename
                class_label = class_labels.get(class_name, -1)  # Get class label
                
                if class_label != -1:
                    mask = np.array(load_img(mask_path, color_mode="grayscale"))
                    mask = (mask > 0).astype(np.uint8)  # Convert mask to binary
                    masks.append(mask)
                    
                    img = img_to_array(load_img(img_path)) / 255.0  # Normalize image
                    images.append(img)
    
    if not images or not masks:
        raise ValueError("No images or masks found in the specified directories.")
    
    return np.array(images), np.array(masks)

In [4]:
# Load images and masks
images, masks = load_data(images_dir, masks_dir)

# Print shapes of images and masks to check if they are loaded correctly
print("Number of images:", len(images))
print("Number of masks:", len(masks))

Number of images: 88
Number of masks: 88


In [5]:
# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(images, masks, test_size=0.2, random_state=42)

In [6]:
# Define model architecture (U-Net) with reduced number of filters
def unet_model(input_shape=(None, None, 3)):
    inputs = tf.keras.Input(shape=input_shape)
    
    # Encoder
    conv1 = layers.Conv2D(32, 3, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(32, 3, activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = layers.Conv2D(64, 3, activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)
    
    # Decoder
    up1 = layers.Conv2DTranspose(32, 2, strides=(2, 2), padding='same')(conv2)
    concat1 = layers.concatenate([conv1, up1], axis=3)
    conv3 = layers.Conv2D(32, 3, activation='relu', padding='same')(concat1)
    conv3 = layers.Conv2D(32, 3, activation='relu', padding='same')(conv3)
    
    outputs = layers.Conv2D(1, 1, activation='sigmoid')(conv3)
    
    return tf.keras.Model(inputs=inputs, outputs=outputs)

In [7]:
# Create U-Net model with reduced complexity
model = unet_model()

In [8]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy')

In [9]:
# Train the model with reduced batch size
history = model.fit(X_train, y_train, epochs=20, batch_size=8, validation_split=0.2)

Epoch 1/20


ResourceExhaustedError: Graph execution error:

Detected at node 'model/conv2d/Relu' defined at (most recent call last):
    File "c:\Users\User\anaconda3\envs\google\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\User\anaconda3\envs\google\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\User\anaconda3\envs\google\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\User\anaconda3\envs\google\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "c:\Users\User\anaconda3\envs\google\lib\site-packages\ipykernel\kernelapp.py", line 737, in start
      self.io_loop.start()
    File "c:\Users\User\anaconda3\envs\google\lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\User\anaconda3\envs\google\lib\a

In [ ]:
# Plot learning curves
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Evaluate the model on test set
y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(np.uint8)

In [ ]:
# Compute IoU for each class
iou_scores = []
for i in range(4):
    intersection = np.sum((y_test == i) & (y_pred_binary == i))
    union = np.sum((y_test == i) | (y_pred_binary == i))
    iou = intersection / union
    iou_scores.append(iou)
    print(f'IoU for class {i}: {iou:.4f}')

In [ ]:
# Calculate overall mean IoU
mean_iou = np.mean(iou_scores)
print(f'Mean IoU: {mean_iou:.4f}')

In [ ]:
# Visualization
n_examples = 5
idx = np.random.randint(0, len(X_test), n_examples)

plt.figure(figsize=(15, 10))
for i in range(n_examples):
    plt.subplot(3, n_examples, i + 1)
    plt.imshow(X_test[idx[i]])
    plt.title('Input Image')
    plt.axis('off')
    
    plt.subplot(3, n_examples, i + n_examples + 1)
    plt.imshow(y_test[idx[i]].squeeze(), cmap='gray')
    plt.title('Ground Truth Mask')
    plt.axis('off')
    
    plt.subplot(3, n_examples, i + 2*n_examples + 1)
    plt.imshow(y_pred_binary[idx[i]].squeeze(), cmap='gray')
    plt.title('Predicted Mask')
    plt.axis('off')
plt.show()